In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy
import xgboost as xgb
import statistics as st

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
valid_df = pd.read_csv("validation.csv")

In [ ]:
test_df = pd.read_csv("test.csv")

In [34]:
def test_results(threshold, y_p, real_clicks):
    # real_clicks = valid_df["click"]
    
    clicks = y_p > threshold
    errs = len(clicks) - sum(clicks == real_clicks)
    num_clicks = len(valid_df[real_clicks == 1]) + sum(clicks)
    ratio = 1-errs/num_clicks
    clicked = (sum(clicks) - sum((clicks - valid_df.click) == 1))
    
    print("Number of predicted clicks ", sum(clicks), " with threshold set to ", threshold)
    print("Number of predictions matching original ", sum(clicks == real_clicks), " -> ", errs, " errors")
    print("Total num of clicks ", num_clicks, " -> ratio ", ratio)
    print("CTR ", clicked/sum(clicks), " and clicks ", clicked)
    
    return clicks

In [5]:
def test_results_budget(clicks, payprice, real_clicks):
    # payprice = valid_df.payprice
    # real_clicks = valid_df.click
    sum_ones = 0
    spent = 0
    click_total = 0

    for i in range(0,len(clicks),3):
        if (clicks[i] == 1) & (spent < 6250):
            click_total = click_total + 1
            spent = spent + payprice[i]
            if real_clicks[i] == 1:
                sum_ones = sum_ones + 1

        if spent > 6250:
            break

    cpc = spent/sum_ones
    ctr_1 = sum_ones/click_total
    print("Sum payed for the clicked ones ", spent, " successful clicks ", sum_ones, " out of ", click_total)
    print("Cost per Click ", cpc)
    print("CTR ", ctr_1)

In [26]:
print("Num of clicks ", sum(train_df.click), " out of ", len(train_df)," auctions which is ", sum(train_df.click)*100/len(train_df), "%")
print("Minimum payed price ", min(train_df.payprice), " and maximum payed price ", max(train_df.payprice))
print("Minimum bid price ", min(train_df.bidprice), " and maximum bid price ", max(train_df.bidprice))
print("Num of bids of each price")
train_df.bidprice.value_counts()

Num of clicks  2034  out of  2697738  auctions which is  0.0753964988446 %
Minimum payed price  0  and maximum payed price  300
Minimum bid price  227  and maximum bid price  300
Num of bids of each price


300    1038847
294     353123
238     350714
227     281102
241     225629
277     200177
254     130893
249     117253
Name: bidprice, dtype: int64

In [27]:
print("CTR is " , sum(train_df.click)/len(train_df))
print("CPM is " , sum(train_df.payprice)*1000/len(train_df))
print("eCPC is " , sum(train_df.payprice)/sum(train_df.click))

CTR is  0.000753964988446
CPM is  80251.0247474
eCPC is  106438.662242


## clean data

In [6]:
for index, c in train_df.iterrows():  
    if c.slotformat in "Na":
        train_df.set_value(index, 'slotformat', 10)
        
d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
train_df['adexchange']=train_df['adexchange'].map(d)

In [ ]:
for index, c in test_df.iterrows():  
    if c.slotformat in "Na":
        test_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
test_df['adexchange']=test_df['adexchange'].map(d)

In [7]:
for index, c in valid_df.iterrows():  
    if c.slotformat in "Na":
        valid_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
valid_df['adexchange']=valid_df['adexchange'].map(d)

## Constant bidding

In [39]:
const = train_df.bidprice.get_values() #[300, 294, 238, 227, 241, 277, 254, 249]
const = np.unique(const)
const = [c+1 for c in cost]
#const = const_range+const_range_upper
const = np.sort(const)
print(const)

[228 239 242 250 255 278 295 301]


doing for training set (ctr) and validation set (ctr_v)

In [44]:
ctr = np.arange(0, len(const), dtype=np.float)
ctr_v = np.arange(0, len(const), dtype=np.float)
i = 0

for c in const:
    filt = train_df.click[train_df.bidprice < c]
    filt_v = valid_df.click[valid_df.bidprice < c]
    ctr[i] = sum(filt)*100/len(filt)
    ctr_v[i] = sum(filt_v)*100/len(filt_v)
    print("Price ", c)
    print("num of clicks: ", sum(filt), "total: ", len(filt))
    print("num of clicks_v: ", sum(filt_v), "total_v: ", len(filt_v), "\n")
    i = i + 1
    
max_index = np.argmax(ctr)
max_index_v = np.argmax(ctr_v)
print("The highest CTR for constant bidding is for ", const[max_index], " with CTR = ", ctr[max_index], "%")
print("The highest CTR for constant bidding  in valid. set is for ", const[max_index_v], " with CTR = ", ctr_v[max_index_v], "%")

Price  228
num of clicks:  276 total:  281102
num of clicks_v:  35 total_v:  31467 

Price  239
num of clicks:  456 total:  631816
num of clicks_v:  52 total_v:  70253 

Price  242
num of clicks:  620 total:  857445
num of clicks_v:  74 total_v:  95351 

Price  250
num of clicks:  651 total:  974698
num of clicks_v:  76 total_v:  108437 

Price  255
num of clicks:  748 total:  1105591
num of clicks_v:  85 total_v:  123073 

Price  278
num of clicks:  1053 total:  1305768
num of clicks_v:  121 total_v:  145272 

Price  295
num of clicks:  1225 total:  1658891
num of clicks_v:  143 total_v:  184528 

Price  301
num of clicks:  2034 total:  2697738
num of clicks_v:  226 total_v:  299749 

The highest CTR for constant bidding is for  228  with CTR =  0.0981850004625 %
The highest CTR for constant bidding  in valid. set is for  228  with CTR =  0.1112276353 %


In [45]:
clicks_const = valid_df.bidprice < const[max_index_v]
test_results_budget(clicks_const, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Random bidding

In [46]:
# 74 is the difference
upper = range(10, 80, 10)

In [47]:
random_bid_basics = np.random.rand(len(train_df), len(upper))
name = ["bid1", "bid2", "bid3", "bid4", "bid5", "bid6", "bid7"]
ctr_r = np.arange(0, len(upper), dtype=np.float)
bottom = 227
i = 0

for u in upper:
    train_df[name[i]] = random_bid_basics[:,i] * u + bottom
   
    
    filt = train_df.click[train_df.bidprice < train_df[name[i]]]
    ctr_r[i] = sum(filt)*100/len(filt)
    print("num of clicks: ", sum(filt), "total: ", len(filt))
    
    i = i+1

num of clicks:  276 total:  281102
num of clicks:  408 total:  506039
num of clicks:  477 total:  667685
num of clicks:  563 total:  776700
num of clicks:  611 total:  843148
num of clicks:  674 total:  920408
num of clicks:  735 total:  990521


In [48]:
max_index = np.argmax(ctr_r)
print("The highest CTR for random bidding is for upper bound at ", upper[max_index], " with CTR = ", ctr_r[max_index], "%")


The highest CTR for random bidding is for upper bound at  10  with CTR =  0.0981850004625 %


In [52]:
random_bid_basics = np.random.rand(len(valid_df), len(upper))
valid_df["rnd_bid"] = random_bid_basics[:,1] * u + bottom

clicks_rand = valid_df.bidprice < valid_df["rnd_bid"]
test_results_budget(clicks_const, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Linear model

In [38]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']


X_train = train_df[features]
y_train = train_df['click']

In [11]:
# Train the model to estimate pCTR
clf = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'binary:logistic')
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [39]:
# preprocess validation set
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid = valid_df[features]

#X_valid['advertiser']=X_valid['advertiser'].astype(str)
#X_valid['adexchange']=X_valid['adexchange'].astype(str)

#X_valid = X_valid.astype("int")


In [ ]:
## predicts probabilities of being clicked for validation
y_p_valid = clf.predict(X_valid)

avgCTR is the average of probability that the ad will be clicked

In [ ]:
avgCTR = sum(y_p_valid)/len(y_p_valid)
print("Average CTR ", avgCTR)

In [ ]:
def calculate_bid(base_bid, y_pred, avgCTR):
    bid = np.multiply(y_pred, base_bid/avgCTR)
    return bid

In [ ]:
def max_bid_base(prices, y_p_valid, avgCTR):
    ration = np.zeros(len(prices))
    total = len(valid_df)

    for i in range(len(prices)):
        tmp = calculate_bid(prices[i], y_p_valid, avgCTR) # calculates our bid price
        tmp_2 = valid_df.click[valid_df.bidprice < tmp] # stores 0/1 for those auctions we won
        if len(tmp_2) == 0:
            ration[i] = 0
        else:
            ration[i] = sum(tmp_2)/len(tmp_2) # clicked/won actions

    print("The bid_base with best CTR: ", prices[ration.argmax()])
    print("CTR for winning bid_base is ", ration.max())
    return prices[ration.argmax()]

In [ ]:
prices = range(1, 450, 1)

bid_base = max_bid_base(prices, y_p_valid, avgCTR)

In [ ]:
tmp = calculate_bid(bid_base, y_p_valid, avgCTR)
tmp_2 = valid_df.payprice[valid_df.bidprice < tmp]

print("Cost for valid set ", sum(tmp_2))

In [ ]:
our_bids = calculate_bid(bid_base, y_p_valid, avgCTR) # calculates our bid price
price_payed = 0
clicked_won = 0 
won = 0

for i in range(len(our_bids)): 
    if (valid_df.bidprice[i] <= our_bids[i]):
        price_payed = price_payed + valid_df.payprice[i]
        if (price_payed < 6250):       
            won = won + 1
            if (valid_df.click[i] == 1):
                clicked_won = clicked_won + 1
        else:
            break
        
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )

## Best performing model

### prediction with hour and weekday separated

In [9]:
# Train the model to estimate pCTR
clf_0 = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_0.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [13]:
## predicts probabilities of being clicked for validation
y_p_valid_0 = clf_0.predict(X_valid)

threshold = 0.01
clicks = y_p_valid_1 > threshold
errs = len(clicks) - sum(clicks == valid_df['click'])
num_clicks = len(valid_df[valid_df.click == 1]) + sum(clicks)
ratio = 1-errs/num_clicks
print("Number of predicted clicks ", sum(clicks), " with threshold set to ", threshold)
print("Number of predictions matching original ", sum(clicks == valid_df['click']), " -> ", errs, " errors")
print("Total num of clicks ", num_clicks, " -> ratio ", ratio)

In [14]:
threshold_0 = 0.01

clicks_0 = test_results(threshold_0, y_p_valid_0, valid_df["click"])

Number of predicted clicks  617  with threshold set to  0.01
Number of predictions matching original  298918  ->  831  errors
Total num of clicks  843  -> ratio  0.0142348754448


In [26]:
test_results_budget(clicks_0, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6269  successful clicks  1  out of  73
Cost per Click  6269.0
CTR  0.0136986301369863


### prediction for weekday, hour and weekday+hour

In [40]:
# encoding weekday and hour into one variable
X_train['weekday']=X_train['weekday'].astype(str)
X_train['hour']=X_train['hour'].astype(str)

X_train['time'] = X_train[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train = X_train.astype("int")

#X_train = X_train.drop(["hour", "weekday"], axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [41]:
# encoding weekday and hour into one variable
X_valid['weekday']=X_valid['weekday'].astype(str)
X_valid['hour']=X_valid['hour'].astype(str)

X_valid['time'] = X_valid[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid = X_valid.astype("int")

#X_valid = X_valid.drop(["hour", "weekday"], axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [42]:
# Train the model to estimate pCTR
clf_1 = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [43]:
## predicts probabilities of being clicked for validation
y_p_valid_1 = clf_1.predict(X_valid)

In [44]:
threshold_1 = 0.01

clicks_1 = test_results(threshold_1, y_p_valid_1, valid_df["click"])

Number of predicted clicks  426  with threshold set to  0.01
Number of predictions matching original  299107  ->  642  errors
Total num of clicks  652  -> ratio  0.0153374233129
CTR  0.0117370892019  and clicks  5


In [45]:
test_results_budget(clicks_1, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6412  successful clicks  3  out of  67
Cost per Click  2137.33333333
CTR  0.04477611940298507


## XGBoost chosen atributes

In [16]:
#Define features
features_a = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']

X_train_a = train_df[features_a]              
              
X_train_a['weekday']=X_train_a['weekday'].astype(str)
X_train_a['hour']=X_train_a['hour'].astype(str)

X_train_a['time'] = X_train_a[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train_a = X_train_a.astype("int")
X_train_a = X_train_a.drop(["hour", "weekday"], axis=1)

y_train_a = train_df['click']

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [17]:
clf_a = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_a.fit(X_train_a, y_train_a)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [18]:
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid_a = valid_df[features_a]              
              
X_valid_a['weekday']=X_valid_a['weekday'].astype(str)
X_valid_a['hour']=X_valid_a['hour'].astype(str)

X_valid_a['time'] = X_valid_a[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid_a = X_valid_a.astype("int")
X_valid_a = X_valid_a.drop(["hour", "weekday"], axis=1)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [19]:
y_p_valid_a = clf_a.predict(X_valid_a)

In [46]:
clicks = test_results(0.01, y_p_valid_a, valid_df.click)

Number of predicted clicks  528  with threshold set to  0.01
Number of predictions matching original  298999  ->  750  errors
Total num of clicks  754  -> ratio  0.0053050397878
CTR  0.00378787878788  and clicks  2


In [47]:
test_results_budget(clicks, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6311  successful clicks  1  out of  70
Cost per Click  6311.0
CTR  0.014285714285714285


## Undersampling

In [12]:
ratio_1_0 = sum(train_df.click)/len(train_df)
print("Percentage of 1s in dataset train ", ratio_1_0*100)
subset = train_df.sample(frac=0.001)
subset = pd.concat([subset, train_df[train_df.click == 1]])

Percentage of 1s in dataset train  0.0753964988446


In [13]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y


X_train_us = subset[features]
y_train_us = subset['click']

# encoding weekday and hour into one variable
X_train_us['weekday']=X_train_us['weekday'].astype(str)
X_train_us['hour']=X_train_us['hour'].astype(str)

X_train_us['time'] = X_train_us[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train_us = X_train_us.astype("int")

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [42]:
clf_us = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_us.fit(X_train_us, y_train_us)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [14]:
X_valid_us = valid_df[features]

# encoding weekday and hour into one variable
X_valid_us['weekday']=X_valid_us['weekday'].astype(str)
X_valid_us['hour']=X_valid_us['hour'].astype(str)

X_valid_us['time'] = X_valid_us[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid_us = X_valid_us.astype("int")

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [43]:
y_predict_us = clf_1.predict(X_valid_us)

In [44]:
threshold_us = 0.01

clicks_us = test_results(threshold_us, y_predict_us, valid_df["click"])

Number of predicted clicks  426  with threshold set to  0.01
Number of predictions matching original  299107  ->  642  errors
Total num of clicks  652  -> ratio  0.0153374233129


In [45]:
test_results_budget(clicks_us, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6412  successful clicks  3  out of  67
Cost per Click  2137.33333333
CTR  0.04477611940298507


## Undersampled RF

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random

In [7]:
# Set the random seed
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

random_forest = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)

In [8]:
random_forest.fit(X=X_train_us, y=y_train_us)

NameError: name 'X_train_us' is not defined

In [18]:
y_rf = random_forest.predict(X_valid_us)

In [20]:
clicks_rf = test_results(0.5, y_rf, valid_df.click)

Number of predicted clicks  80124  with threshold set to  0.5
Number of predictions matching original  219587  ->  80162  errors
Total num of clicks  80350  -> ratio  0.00233976353454


In [21]:
test_results_budget(clicks_rf, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6324  successful clicks  0  out of  63
Cost per Click  inf
CTR  0.0


### Predicting payprice

In [22]:
#Define features

X_train_pp = pd.concat([X_train, train_df["click"]], axis=1)
y_train_pp = train_df['payprice']

X_valid_pp = pd.concat([X_valid, valid_df["click"]], axis=1)

In [23]:
# Train the model to estimate pCTR
clf_pp = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'reg:linear')
clf_pp.fit(X_train_pp, y_train_pp)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [24]:
y_pp = clf_pp.predict(X_valid_pp)

In [53]:
valid_pp = valid_df
valid_pp["predicted_pp"] = y_pp
valid_pp["predicted_click"] = clicks_1

NameError: name 'clicks_1' is not defined

In [ ]:
valid_pp_sorted = valid_pp.sort_values("predicted pp")

In [ ]:
sum_ones = 0
spent = 0
click_total = 0

for index, i in valid_pp_sorted.iterrows():
    if (valid_pp_sorted.predicted_click[index] == 1) & (spent < 6250) & (valid_pp_sorted.predicted_pp[index] > 54):
        click_total = click_total + 1
        spent = spent + valid_pp_sorted.payprice[index]
        if valid_pp_sorted.click[index] == 1:
            sum_ones = sum_ones + 1

    if spent > 6250:
        break
        
cpc = spent/sum_ones
ctr_1 = sum_ones/click_total
print("Sum payed for the clicked ones ", spent, " successful clicks ", sum_ones, " out of ", click_total)
print("Cost per Click ", cpc)
print("CTR ", ctr_1)
    

### Create new column for cheapest clicks and drop hour & weekday

In [ ]:
med = np.median(train_df.payprice[train_df.click == 1])

train_df_cc = train_df
X_train_cc = X_train.
train_df_cc["cheapest_clicks"] = train_df_cc.payprice < med
y_train_cc = train_df_cc.click == 1
y_train_cc = y_train_cc.astype("int")

In [ ]:
clf_cc = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'reg:linear')
clf_cc.fit(X_train_cc, y_train_cc)

In [ ]:
y_cc = clf_cc.predict(X_valid_cc)

In [ ]:
threshold_cc = 0.005
clicks_cc = y_cc > threshold_cc
errs_cc = len(clicks_cc) - sum(clicks_cc == valid_df['click'])
num_clicks_cc = len(valid_df[valid_df.click == 1]) + sum(clicks_cc)
ratio_cc = 1-errs_cc/num_clicks_cc
print("Number of predicted clicks ", sum(clicks_cc), " with threshold set to ", threshold_cc)
print("Number of predictions matching original ", sum(clicks_cc == valid_df['click']), " -> ", errs_cc, " errors")
print("Total num of clicks ", num_clicks_cc, " -> ratio ", ratio_cc)

In [ ]:
sum(clicks == clicks_cc)

In [ ]:
X_valid_cc = X_valid.drop(["weekday", "hour"], axis=1)

## RF on validation set

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random

In [8]:
# Set the random seed
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

random_forest = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)

In [9]:
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]

valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid = valid_df[features]

# encoding weekday and hour into one variable
X_valid['weekday']=X_valid['weekday'].astype(str)
X_valid['hour']=X_valid['hour'].astype(str)

X_valid['time'] = X_valid[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid = X_valid.astype("int")

#X_valid = X_valid.drop(["hour", "weekday"], axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [10]:
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
train_df['area'] = train_df['slotwidth']*train_df['slotheight']

X_train = train_df[features]

# encoding weekday and hour into one variable
X_train['weekday']=X_train['weekday'].astype(str)
X_train['hour']=X_train['hour'].astype(str)

X_train['time'] = X_train[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train = X_train.astype("int")

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [11]:
y_valid = valid_df["click"]

random_forest.fit(X=X_valid, y=y_valid)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [12]:
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange", "time"]

for feature, importance in zip(features, random_forest.feature_importances_):
    print (feature, ":", importance)

weekday : 0.0421076751229
hour : 0.0947864476496
region : 0.102637393286
city : 0.354731323184
area : 0.0809471014576
slotprice : 0.0936563767411
advertiser : 0.0875729759374
adexchange : 0.0167483203428
time : 0.126812386278


In [17]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299749 entries, 0 to 299748
Data columns (total 9 columns):
weekday       299749 non-null int64
hour          299749 non-null int64
region        299749 non-null int64
city          299749 non-null int64
area          299749 non-null int64
slotprice     299749 non-null int64
advertiser    299749 non-null int64
adexchange    299749 non-null int64
time          299749 non-null int64
dtypes: int64(9)
memory usage: 22.9 MB


In [12]:
print ("OOB accuracy is ", random_forest.oob_score_)

OOB accuracy is  0.999079229622


In [ ]:
y_pred = random_forest.predict(X_valid)

In [ ]:
sum(abs(y_valid - y_pred))

In [ ]:
sum(y_valid)

In [1]:
y_p_train = random_forest.predict(X_train.head(500000))

NameError: name 'random_forest' is not defined

In [ ]:
sum(y_p_train)

In [ ]:
false_positive = sum((y_p_train - train_df.click.head(100000)) == 1)
false_negative = sum((y_p_train - train_df.click.head(100000)) == -1)
print("Clicks marked as zero ", false_negative, " non-clickes marked as one ", false_positive)
                     

In [ ]:
sum(y_p_train == train_df.click.head(100000))